In [1]:
import pandas as pd
import numpy as np
import time
import requests
import matplotlib.pyplot as plt
import terminalLogic
from terminalLogic import Algo
import client
from client import engine
from typing import Optional, Dict, Any, List
from requests import Request, Session, Response
import sys
import hmac
import urllib.parse
import json
import plotly.graph_objects as go
from datetime import datetime
plt.rcParams["figure.figsize"] = (100,10)


In [2]:
!jt -t onedork

In [3]:
df = engine.getData('BTC-PERP')
df['time'] = pd.to_datetime(df['time'], unit='ms')
df = df[::-1]
df

,time,open,high,low,close,volume
1499,2021-10-30 22:25:00,61471.0,61471.0,61459.0,61468.0,36963.7485
1498,2021-10-30 22:24:00,61485.0,61511.0,61463.0,61471.0,100525.0103
1497,2021-10-30 22:23:00,61459.0,61488.0,61431.0,61485.0,176117.7548
1496,2021-10-30 22:22:00,61492.0,61494.0,61454.0,61460.0,75390.5114
1495,2021-10-30 22:21:00,61524.0,61542.0,61492.0,61492.0,112336.3565
...,...,...,...,...,...,...
4,2021-10-29 21:30:00,62188.0,62216.0,62186.0,62210.0,409160.3999
3,2021-10-29 21:29:00,62318.0,62318.0,62182.0,62188.0,277463.3588
2,2021-10-29 21:28:00,62320.0,62320.0,62300.0,62318.0,365262.5590
1,2021-10-29 21:27:00,62300.0,62317.0,62300.0,62317.0,29606.3856


In [4]:
df = Algo.fullstate(df,df)
df

,time,open,high,low,close,volume,9-min,21-min,signal,market_returns,trend_returns,entry,ma,ratio,position,entryR,range_returns,sign
1478,2021-10-30 22:04:00,61539.0,61543.0,61534.0,61535.0,10854.9164,61463.111111,61466.428571,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,2021-10-30 22:03:00,61580.0,61580.0,61537.0,61539.0,316180.9884,61472.666667,61469.619048,1.0,0.000065,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1476,2021-10-30 22:02:00,61586.0,61590.0,61575.0,61580.0,82748.7724,61482.666667,61472.857143,1.0,0.000666,0.000065,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1475,2021-10-30 22:01:00,61583.0,61597.0,61577.0,61586.0,4188.0341,61497.444444,61477.380952,1.0,0.000097,0.000666,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1474,2021-10-30 22:00:00,61480.0,61607.0,61478.0,61583.0,227626.2845,61513.222222,61483.380952,1.0,-0.000049,0.000097,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-10-29 21:30:00,62188.0,62216.0,62186.0,62210.0,409160.3999,62279.000000,62361.000000,-1.0,-0.000755,-0.000000,0.0,62279.000000,0.998892,1.0,0.0,-0.000755,1.0
3,2021-10-29 21:29:00,62318.0,62318.0,62182.0,62188.0,277463.3588,62266.888889,62351.142857,-1.0,-0.000354,0.000755,0.0,62266.888889,0.998733,1.0,0.0,-0.000354,1.0
2,2021-10-29 21:28:00,62320.0,62320.0,62300.0,62318.0,365262.5590,62254.444444,62340.571429,-1.0,0.002088,0.000354,0.0,62254.444444,1.001021,1.0,0.0,0.002088,-1.0
1,2021-10-29 21:27:00,62300.0,62317.0,62300.0,62317.0,29606.3856,62256.666667,62338.047619,-1.0,-0.000016,-0.002088,0.0,62256.666667,1.000969,1.0,0.0,-0.000016,-1.0


In [5]:
y = Algo.folio(df)
y

C:\Users\tbrid\Desktop\terminalLogic\code\terminalLogic.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self['rangeSign'] = np.where(self['range_returns'] < 0, 'neg','pos')
C:\Users\tbrid\Desktop\terminalLogic\code\terminalLogic.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self['trendSign'] = np.where(self['trend_returns'] < 0, 'neg','pos')
C:\Users\tbrid\Desktop\terminalLogic\code\terminalLogic.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,time,open,high,low,close,volume,9-min,21-min,signal,market_returns,...,entryR,range_returns,sign,just_date,rangeSign,trendSign,rangeSeries,trendSeries,rangePairs,trendPairs
1396,2021-10-30 20:42:00,61744.0,61796.0,61743.0,61762.0,2.383379e+05,61844.666667,61776.142857,1.0,-0.000664,...,0.0,0.000664,-1.0,2021-10-30,pos,neg,5,6,1,0
1395,2021-10-30 20:41:00,61791.0,61791.0,61743.0,61744.0,3.681250e+06,61837.333333,61778.523810,1.0,-0.000291,...,0.0,0.000291,-1.0,2021-10-30,pos,neg,5,6,1,0
1394,2021-10-30 20:40:00,61789.0,61819.0,61786.0,61791.0,3.026308e+05,61827.555556,61779.666667,1.0,0.000761,...,0.0,-0.000761,1.0,2021-10-30,neg,neg,6,6,1,0
1364,2021-10-30 20:10:00,61912.0,61913.0,61850.0,61876.0,4.891696e+05,61852.888889,61848.333333,1.0,0.000711,...,0.0,0.000711,-1.0,2021-10-30,pos,pos,15,17,5,1
1363,2021-10-30 20:09:00,61904.0,61913.0,61903.0,61913.0,1.067182e+05,61857.777778,61849.380952,1.0,0.000598,...,0.0,0.000598,1.0,2021-10-30,pos,pos,15,17,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2021-10-29 21:56:00,62426.0,62426.0,62410.0,62410.0,1.420721e+05,62381.777778,62372.142857,1.0,0.000048,...,0.0,-0.000048,1.0,2021-10-29,neg,pos,650,671,131,29
29,2021-10-29 21:55:00,62410.0,62426.0,62407.0,62426.0,2.069200e+05,62384.777778,62374.000000,1.0,0.000256,...,0.0,-0.000256,1.0,2021-10-29,neg,pos,650,671,131,29
24,2021-10-29 21:50:00,62371.0,62414.0,62371.0,62410.0,8.529278e+04,62415.000000,62385.476190,1.0,-0.000112,...,0.0,0.000112,-1.0,2021-10-29,pos,neg,651,674,132,29
23,2021-10-29 21:49:00,62384.0,62384.0,62371.0,62371.0,4.707915e+04,62420.444444,62387.142857,1.0,-0.000625,...,0.0,0.000625,-1.0,2021-10-29,pos,neg,651,674,132,29


In [6]:
df['just_date'] = df['time'].dt.date
df['just_date']

1478    2021-10-30
1477    2021-10-30
1476    2021-10-30
1475    2021-10-30
1474    2021-10-30
           ...    
4       2021-10-29
3       2021-10-29
2       2021-10-29
1       2021-10-29
0       2021-10-29
Name: just_date, Length: 1479, dtype: object

In [7]:
#column for negative and positive
dfR=df.dropna()
dfR['sign'] = np.where(dfR['range_returns'] < 0, 'neg','pos')
dfT['sign'] = np.where(dfT['trend_returns'] < 0, 'neg','pos')

#consecutive groups
dfR['series'] = dfR['sign'].ne(dfR['sign'].shift()).cumsum()
dfT['series'] = dfT['sign'].ne(dfT['sign'].shift()).cumsum()

#removed groups with length more like 2
df1 = dfR[dfR['series'].map(dfR['series'].value_counts()).gt(2)]
df2 = dfT[dfT['series'].map(dfT['series'].value_counts()).gt(2)]

#tested if order `pos-neg` of groups, if not removed groups
m1 = df1['sign'].eq('pos') & df1['sign'].shift(-1).eq('neg')
m2 = df1['sign'].eq('neg') & df1['sign'].shift().eq('pos')
m3 = df2['sign'].eq('pos') & df2['sign'].shift(-1).eq('neg')
m4 = df2['sign'].eq('neg') & df2['sign'].shift().eq('pos')
groupsR = df1.loc[m1 | m2, 'series']
df1 = df1[df1['series'].isin(groupsR)].copy()
df1['pairs'] = (df1['sign'].ne(df1['sign'].shift()) & df1['sign'].eq('pos')).cumsum()
groupsT = df2.loc[m3 | m4, 'series']
df2 = df2[df2['series'].isin(groupsT)].copy()
df2['pairs'] = (df2['sign'].ne(df2['sign'].shift()) & df2['sign'].eq('pos')).cumsum()
rangeTradeCounts = df1['series'].nunique()
trendTradeCounts = df2['series'].nunique()
totalTrades = rangeTradeCounts + trendTradeCounts
df1






dfT=df.dropna()
dfT['sign'] = np.where(dfT['trend_returns'] < 0, 'neg','pos')
#consecutive groups
dfT['series'] = dfT['sign'].ne(dfT['sign'].shift()).cumsum()
#removed groups with length more like 2
df2 = dfT[dfT['series'].map(dfT['series'].value_counts()).gt(2)]
#tested if order `pos-neg` of groups, if not removed groups
m3 = df2['sign'].eq('pos') & df2['sign'].shift(-1).eq('neg')
m4 = df2['sign'].eq('neg') & df2['sign'].shift().eq('pos')
groupsT = df2.loc[m3 | m4, 'series']
df2 = df2[df2['series'].isin(groupsT)].copy()
df2['pairs'] = (df2['sign'].ne(df2['sign'].shift()) & df2['sign'].eq('pos')).cumsum()
rangeTradeCounts = df1['series'].nunique()
trendTradeCounts = df2['series'].nunique()
totalTrades = rangeTradeCounts + trendTradeCounts
df1
# def traderets():
#     trades = df1['series'].nunique()
#     tradeValues = df1['range_returns'].sum()
#     valuePerTrade = df1.drop
# R = df['series'].dropna().nunique()
# W = len(df.loc[df.range_returns.dropna() > 0])
# L = len(df.loc[df.range_returns.dropna() < 0])
# P = 2*W*L

# Z_score = (N*(R-0.5)-P)/((P*(P-N))/(N-1))**(1/2)
# Z_score


<ipython-input-7-aab05113daa6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfR['sign'] = np.where(dfR['range_returns'] < 0, 'neg','pos')


NameError: name 'dfT' is not defined

### I need to sum all the values for each row in dataframe['range_returns'] where all of the values in dataframe['series'] are the same value. I would like to output a new df with columns ['seriesID', 'series_returns'].


### Thank you in advance

In [ ]:
dataframe = df1.filter(['just_date', 'close', 'range_returns', 'sign', 'series', 'pairs'], axis=1)
# find % pos and neg trades
dataframe.head(20)

In [ ]:
import re
seriesRets = pd.DataFrame(columns=['seriesID', 'sign', 'seriesRets'])
seriesRets['seriesRets'] = dataframe.groupby(['series', 'sign'])['range_returns'].transform('sum')
seriesRets['seriesID'] = dataframe['series']
seriesRets['sign'] = dataframe['sign']
seriesRets = seriesRets.drop_duplicates()
total = seriesRets['seriesRets'].sum()
total = total.astype('float64')
totes = float("{:.8f}".format(total))
tradeCount = seriesRets['seriesID'].nunique()
meanTrade = totes / tradeCount
st_dev = seriesRets['seriesRets'].std()
score = seriesRets['seriesRets'].values
zScore = (score - meanTrade) / st_dev
seriesRets['ZScore'] = zScore
seriesRets['win_lose'] = np.where(seriesRets['sign'] == 'pos', 1, -1)
winCount = []
loseCount = []
totalCount = []
for win in seriesRets['win_lose']:
    if win == 1:
        winCount.append(1)
        totalCount.append(len(winCount))
    else:
        loseCount.append(1)
        totalCount.append(len(loseCount))
seriesRets['tradeID'] = totalCount
winners = seriesRets[seriesRets['win_lose']== 1]


In [ ]:
trade = {'Algo':['Range'],
         'tradeCount':[tradeCount],
         'total':[totes],
         'mean':[meanTrade],
         'st_dev':[st_dev],
         'Zscore':[zScore]
          }
tradeDf = pd.DataFrame(trade)
# seriesRets['win_lose'] = np.where(seriesRets['sign'] == 'pos', 1, -1)
# seriesRets.groupby('sign').win_lose.value_counts().unstack()
# winningTradeCount
tradeDf

In [ ]:
newSeries = seriesRets[::-1]
wins=0
lose=0
winCount = []
loseCount = []
totalCount = []
for win in newSeries['win_lose']:
    if win == 1:
        wins+=1
        winCount.append(wins)
        loseCount.append(np.nan)
    else:
        lose+=1
        loseCount.append(lose)
        winCount.append(np.nan)
newSeries['winningTradeID'] = winCount
newSeries['losingTradeID'] = loseCount
newSeries['winningTradeID'].max()
overview = pd.DataFrame(columns=['TotalTradesTaken','WinningTrades',
                                 'MaxWin', 'MinWin', 'LosingTrades',
                                 'MaxLoss', 'MinLoss', 'ZScore',
                                'SharpeRatio', 'Win/LossRatio',
                                '30DayPnL'])
overview['TotalTradesTaken'] = totalCount
overview['LosingTrades'] = lose
overview['WinningTrades'] = wins
overview['MaxWin'] = newSeries['seriesRets'].max()
overview['MaxLoss'] = newSeries['seriesRets'].min()



overview

In [ ]:
overview = pd.DataFrame(columns=['TotalTradesTaken','WinningTrades',
                                 'MaxWin', 'MinWin', 'LosingTrades',
                                 'MaxLoss', 'MinLoss', 'ZScore',
                                'SharpeRatio', 'Win/LossRatio',
                                '30DayPnL'])
overview['TotalTradesTaken'] = totalCount
overview['LosingTrades'] = lose
overview['WinningTrades'] = wins
overview['MaxWin'] = newSeries['seriesRets'].max()
overview['MaxLoss'] = newSeries['seriesRets'].min()



overview

In [ ]:
maxwin = newSeries['seriesRets'].max()
maxLoss = newSeries['seriesRets'].min()
totesTaken = newSeries['seriesRets'].value_counts()
numWin = newSeries['winningTradeID'].dropna().value_counts()
numLoss = newSeries['losingTradeID'].dropna().value_counts()
overview

In [ ]:
# #column for negative and positive
# df.dropna()
# df['sign'] = np.where(df['trend_returns'] < 0, 'neg','pos')
# #consecutive groups
# df['series'] = df['sign'].ne(df['sign'].shift()).cumsum()
# #removed groups with length more like 2
# df1 = df[df['series'].map(df['series'].value_counts()).gt(2)]
# #tested if order `pos-neg` of groups, if not removed groups
# m1 = df1['sign'].eq('pos') & df1['sign'].shift(-1).eq('neg')
# m2 = df1['sign'].eq('neg') & df1['sign'].shift().eq('pos')
# groups = df1.loc[m1 | m2, 'series']
# df1 = df1[df1['series'].isin(groups)].copy()
# df1['pairs'] = (df1['sign'].ne(df1['sign'].shift()) & df1['sign'].eq('pos')).cumsum()

# A2 = len(groups)
# A3=A1+A2
# A3

In [ ]:
# Set initial capital
initial_capital = float(22000.0)

# Create df positions
positions = pd.DataFrame(index=df.time.index).fillna(0.0)


# Buy 2 BTC
positions['BTCPERP'] = 1*df['signal']

# Initilize portfolio w value owned
portfolio = positions.multiply(df['close'], axis=0)

# Store diff in shares owned
pos_diff = positions.diff()

# Add 'holdings' to portfolio
portfolio['holdings'] = (positions.multiply(df['close'], axis=0)).sum(axis=1)

# Add 'cash' to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(df['close'], axis=0)).sum(axis=1).cumsum()

# Add 'total' to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add 'returns' to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

portfolio['time'] = df['just_date']
p = portfolio[-1:]
p.drop(columns=['time'], inplace=True)
p = p.reset_index(drop=True)
p.to_json('../web/templates/portfolio2.json', orient='records')
p

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
plt.figure(figsize=(30, 10))
x = portfolio.iloc[-500:]['time']
y = portfolio.iloc[-500:]['total']
values = range(len(x))
plt.xticks(fontsize=22, color="white")
plt.xlabel('Time', color='white',fontsize=22)
plt.yticks(fontsize=22, color='white')
plt.ylabel('Value', color='white',fontsize=22)
plt.plot(values,y)
plt.savefig('../web/assets/portfolioStandings.png')
plt.show()

In [ ]:
portfolio.dropna().to_csv("../web/assets/portfolio.csv")

In [ ]:
plt.plot(np.exp(df['returns'].tail(5000).dropna()).cumprod(), label='Buy/Hold')
plt.plot(np.exp(df['strat_return'].tail(5000).dropna()).cumprod(), label='Strategy')
plt.xticks(rotation=90)
plt.legend();

In [ ]:
# # final value for each column

# print("Returns over time period specified within api call: ")
# marketR = np.exp(df['returns'].dropna()).cumprod().iloc[-1]-1
# stratR = np.exp(df['strat_return'].dropna()).cumprod().iloc[-1]-1
# percentageM = "{:.0%}".format(marketR)
# percentageS = "{:.0%}".format(stratR)
# print('Market: ', percentageM, 'Strat: ', percentageS)
# final value for each column
print("Returns over time period specified within api call: ")
print(np.exp(df['returns'].tail(5000).dropna()).cumprod().iloc[-1] -1)
print(np.exp(df['strat_return'].tail(5000).dropna()).cumprod().iloc[-1] -1)

In [ ]:
portfolio.to_csv("../web/assets/portfolio")

In [ ]:

plt.rcParams["figure.figsize"] = (15,5)

x = portfolio['startTime']
y = portfolio['total']
values = range(len(x))
plt.plot(values,y)


# Show the plot
plt.show()

In [ ]:
# df = add_all_ta_features(
#     df, open="open", high="high", low="low", close="close", volume="volume")
# df